**API Key**

In [ ]:
API_KEY = "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"

**Installation**

In [ ]:
!pip install groq

In [ ]:
!pip install faiss-cpu

In [ ]:
!pip install sentence-transformers


Using  **LLAMA Model**  and  **API Key**  for Chat with Application

In [ ]:
import os

from groq import Groq

client = Groq(
    api_key= API_KEY,
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama3-8b-8192",
)

print(chat_completion.choices[0].message.content)

Reading **Trained Dataset** by using Pandas

**Movies Dataset:** https://docs.google.com/spreadsheets/d/1RXRuXlni1vzXgm0Egl6MysTHMM2zJR7bl8TKTRGnxfQ/edit?usp=sharing

In [ ]:
import pandas as pd

# Load your dataset into a DataFrame
pd = pd.read_csv("/content/Hydra-Movie-Scrape - Hydra-Movie-Scrape (1).csv")

# Use the head() method on the DataFrame to display the first 6 rows
df = pd.head(6)

# Print the DataFrame
print(df)


**Application Generation**

In [ ]:
import pandas as pd
import numpy as np
from groq import Groq
import faiss
from sentence_transformers import SentenceTransformer

# Step 1: Initialize the Groq API client with your API key

client = Groq(api_key=API_KEY)

# Step 2: Load your dataset
data = pd.read_csv("/content/Hydra-Movie-Scrape - Hydra-Movie-Scrape (1).csv")

# Step 3: Handle missing values in the "Summary" column by replacing NaN with empty strings
data['Summary'].fillna('', inplace=True)

# Step 4: Initialize the embedding model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Step 5: Embed the "Summary" column
embeddings = embedder.encode(data['Summary'].tolist(), show_progress_bar=True)

# Step 6: Store the embeddings as a list of arrays in the DataFrame
data['summary_embeddings'] = list(embeddings)

# Step 7: Initialize FAISS index for vector search
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))  # Add embeddings to FAISS

# Step 8: Define a function to search for similar summaries using FAISS
def search_similar_summaries(query, top_n=5):
    query_embedding = embedder.encode([query])
    distances, indices = index.search(query_embedding, top_n)
    return data.iloc[indices[0]]

# Step 9: Define a function to generate the augmented response using the Groq API
def generate_response(user_query):
    # Retrieve similar movie summaries from the dataset
    retrieved_data = search_similar_summaries(user_query, top_n=5)

    # Create context from the retrieved data to pass as input to the Groq API
    context = ""
    for idx, row in retrieved_data.iterrows():
        context += f"Title: {row['Title']}\nSummary: {row['Summary']}\nGenres: {row['Genres']}\nDirector: {row['Director']}\n\n"

    # Construct the final prompt for Groq's language model
    prompt = f"The user asked: {user_query}\nHere are some relevant movies:\n{context}"

    # Use the Groq API to get a completion from the language model
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama3-8b-8192",
    )

    # Access the generated response using dot notation
    generated_text = chat_completion.choices[0].message.content

    return generated_text


# Step 10: Example usage of the RAG application
user_query = "Man of steel"
response = generate_response(user_query)

# Output the generated response
print(response)
